<a href="https://colab.research.google.com/github/Rogerio-mack/fNIRS/blob/main/A_Carol_check_stimulus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verifica e Gera Lista de Arquivos somente com os Estímulos Completos

Isto é $ 3 \times 8  = 24 $ estímulos.

## Install & import `nilearn` and `mne-nirs`

In [1]:
#@markdown
!pip install nilearn
!pip install -U --no-deps https://github.com/mne-tools/mne-python/archive/main.zip
!pip install mne-nirs

from mne.io import read_raw_nirx
from mne.preprocessing.nirs import (optical_density, beer_lambert_law,
                                    temporal_derivative_distribution_repair)
import mne 



     |████████████████████████████████| 9.6 MB 3.9 MB/s 
     |████████████████████████████████| 38.1 MB 3.8 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     / 61.5 MB 663 kB/s
  Created wheel for mne: filename=mne-1.1.dev0-py3-none-any.whl size=7422442 sha256=9951ec16204170e809f6b52ffcfb39a147b38e9b896e5758a9c20753317ad01d
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mazqoo0/wheels/66/fc/d2/9d32190e71b475f1cabf28474c2cbc9274afb98ff6fa2dc7fd
Successfully built mne
     |████████████████████████████████| 93 kB 1.1 MB/s 


## Mount Google Drive

In [2]:
#@markdown
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Common Imports

In [3]:
#@markdown 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns

import h5py
from itertools import compress
from sklearn.preprocessing import scale
from google.colab import widgets

## Routines

In [4]:
#@markdown `def clone_snirf(path_in, path_out, fname='out.snirf')`
def clone_snirf(path_in, path_out, fname='out.snirf'):
  
  if path_in == path_out:
    print('Error: path_in and path_out can not be the same.') 
    return

  if not os.path.exists(path_out):
      os.mkdir(path_out)
      print("Directory " , path_out ,  "was created")
  else:    
      print("Warning: directory " , path_out ,  " already exists")

  if os.path.exists(path_out + fname):
    os.remove(path_out + fname)
    print('Warning: previous output file was deleted.')
  
  print('Input snirf file: ' + path_in + fname)

  fs = h5py.File(path_in + fname,'r')
  fd = h5py.File(path_out + fname,'w')

  if list(fs.keys()).count('formatVersion') >= 1:
    fs.copy('formatVersion',fd,'formatVersion') 
  else:
    print('Warning: there is no formatVersion key in snirf input.')

  if list(fs.keys()).count('nirs') >= 1:
    fs.copy('nirs',fd,'nirs') 
  else:
    print('Error: Invalid snirf file. There is no nirs key in snirf input.')  
  
  print('Output snirf file: ' + path_out + fname)
  
  fd.close()
  fs.close()

  return

# Copia todos arquivos do drive para local

In [5]:
import os
# path_in = '/content/drive/MyDrive/cond 1/snirf/'
path_in = '/content/drive/MyDrive/Cond 2/snirf/'

if 'cond 1' in path_in:
  condicao = 1
else:
  condicao = 2

path_out= '/content/'

In [6]:
# Clear all before copy... 
for f in os.listdir('/content/'):
  if '_run1' in f:
    os.system('rm ' + '/content/' + f )
  if '_run2' in f:
    os.system('rm ' + '/content/' + f )

In [7]:
entries = os.listdir(path_in)
print(entries)

for f in entries:
  if f.count('.snirf') > 0:
    os.system('cp ' + "'" + path_in + f  + "'" + ' ' + '/content/' + f )

['37_run2.snirf', '15_run2.snirf', '32_run2.snirf', '35_run2.snirf', '23_run2.snirf', '33_run2.snirf', '25_run2.snirf', '2_run2.snirf', '38_run2.snirf', '27_run2.snirf', '8_run2.snirf', '30_run2.snirf', '39_run2.snirf', '16_run2.snirf', '36_run2.snirf', '26_run2.snirf', '21_run2.snirf', '12_run2.snirf', '11_run2.snirf', '3_run2.snirf', '28_run2.snirf', '5_run2.snirf', '19_run2.snirf', '1_run2.snirf', '13_run2.snirf', '22_run2.snirf', '29_run2.snirf', '31_run2.snirf', '4_run2.snirf', '10_run2.snirf', '9_run2.snirf', '24_run2.snirf', '14_run2.snirf', '34_run2.snirf', '17_run2.snirf', '18_run2.snirf', '6_run2.snirf']


In [8]:
# for f in os.listdir(path_in):
#  if '_run1' in f:
#    os.system('rm ' + path_in + f )
if path_in == '/content/drive/MyDrive/Cond 2/snirf/':
  for f in os.listdir('/content/'):
    if '_run1' in f:
      os.system('rm ' + '/content/' + f )

if path_in == '/content/drive/MyDrive/cond 1/snirf/':
  for f in os.listdir('/content/'):
    if '_run2' in f:
      os.system('rm ' + '/content/' + f )

In [9]:
entries =  os.listdir(path_out)
entries = [x for x in entries if '.snirf' in x]

# entries

In [10]:
from mne.io import read_raw_nirx
from mne.preprocessing.nirs import (optical_density, beer_lambert_law,
                                    temporal_derivative_distribution_repair)
import mne
import pickle

threshold = 0.60

sci_df_exists = False
sci_df = pd.DataFrame()

raw_haemo_all = {}
epochs_all = {}

for fname in entries:
  raw_intensity = mne.io.read_raw_snirf(path_out + fname, verbose=True)
  raw_intensity.load_data()
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

  #
  # No bad!!!
  #
  # raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < threshold))
  # raw_od.info['bads'] =  bad_list
  
  raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od)
  raw_haemo = raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2,
                              l_trans_bandwidth=0.02)
  print(raw_haemo.filenames)

  events, event_dict = mne.events_from_annotations(raw_haemo)

  reject_criteria = dict(hbo=80e-6)
  tmin, tmax = -5, 15

  epochs = mne.Epochs(raw_haemo, events, event_id=event_dict,
                      tmin=tmin, tmax=tmax,
  #                    reject=reject_criteria, reject_by_annotation=True,
                      proj=True, baseline=(None, 0), preload=True,
                      detrend=None, verbose=True,
                      event_repeated='merge') # ADICIONADO Allowed values are 'error', 'drop', and 'merge'
  # epochs.plot_drop_log()

  raw_haemo_all[fname.split('.')[0]] = raw_haemo
  epochs_all[fname.split('.')[0]] = epochs


Loading /content/37_run2.snirf
Reading 0 ... 21900  =      0.000 ...  2190.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)



<ipython-input-10-b119e44fc3a2>:18: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


('/content/37_run2.snirf',)
Used Annotations descriptions: ['Async1', 'Async2', 'Async3', 'Async4', 'Async5', 'Async6', 'Async7', 'Async8', 'Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 'controle1', 'controle2', 'controle3', 'controle4', 'controle5', 'controle6', 'controle7', 'controle8']
Not setting metadata
24 matching events found
Setting baseline interval to [-5.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 24 events and 201 original time points ...
1 bad epochs dropped
Loading /content/15_run2.snirf


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


Reading 0 ... 21900  =      0.000 ...  2190.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)

('/content/15_run2.snirf',)
Used Annotations descriptions: ['Async1', 'Async2', 'Async3', 'Async4', 'Async5', 'Async6', 'Async7', 'Async8', 'Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 'controle1', 'controle2', 'controle3', 'controle4', 'controle5', 'controle6', 'controle7']


...

('/content/18_run2.snirf',)
Used Annotations descriptions: ['Async1', 'Async2', 'Async3', 'Async4', 'Async5', 'Async6', 'Async7', 'Async8', 'Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 'controle1', 'controle2', 'controle3', 'controle4', 'controle5', 'controle6', 'controle7', 'controle8']
Not setting metadata
24 matching events found
Setting baseline interval to [-5.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 24 events and 201 original time points ...
0 bad epochs dropped
Loading /content/6_run2.snirf


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


Reading 0 ... 21900  =      0.000 ...  2190.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)

('/content/6_run2.snirf',)
Used Annotations descriptions: ['Async1', 'Async2', 'Async3', 'Async4', 'Async5', 'Async6', 'Async7', 'Async8', 'Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 'controle1', 'controle2', 'controle3', 'controle4', 'controle5', 'controle6', 'controle7', 'controle8']
Not setting metadata


<ipython-input-10-b119e44fc3a2>:18: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


24 matching events found
Setting baseline interval to [-5.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 24 events and 201 original time points ...
0 bad epochs dropped


In [11]:
# raw_haemo_all 

In [12]:
# epochs_all

In [13]:
for f in entries: 
  if len(epochs_all[f.split('.')[0]].events) == 24:
    modelo = set(epochs_all[f.split('.')[0]].event_id.keys())
    last_samp = raw_haemo_all[f.split('.')[0]].last_samp/10


In [14]:
def check(subject,detail=False):
  
  if detail:
    print('número de eventos: ', len(epochs_all[subject].events))
    print()
    display(epochs_all[subject])
    print()
    print(epochs_all[subject].events)

    for i in range(len(raw_haemo_all[subject].annotations[:][:])):
      print(raw_haemo_all[subject].annotations[:][i]['description'].ljust(15),raw_haemo_all[subject].annotations[:][i]['onset'])

    print()

  for fname in [subject + '.snirf']: 
    path_out='/content/'

    data = h5py.File(path_out + fname,'r+')

    #
    # create alias for data['nirs']
    #
    for key in data['nirs'].keys():
      exp = key + ' = data["nirs"]["' + key + '"] '
    #  print(exp)
      exec(exp)

  if detail:
    for k in data['nirs'].keys():
      if k.count('stim') == 1:
        print( k, data['nirs'][k]['name'][:] , data['nirs'][k]['data'][:] )

    print()
  
  if len(epochs_all[subject].events) == 24:
    print(subject.ljust(15), 'OK')
    return subject, 'OK'

  diff = modelo - modelo.intersection(epochs_all[subject].event_id.keys())

  if len(epochs_all[subject].events) == 23 and len(diff) == 0:
    if raw_haemo_all[subject].last_samp/10 < data['nirs']['stim16']['data'][0][0] + tmax*10:
      print(subject.ljust(15), 'Tempo do Async8 ', data['nirs']['stim16']['data'][0][0], '+ tmax*10 deve ser menor que último sample ', last_samp)
      return subject, 'Tempo do Async8 + tmax*10 deve ser menor que último sample' 

  if len(diff) >= 1: 
    print(subject.ljust(15), 'Falta eventos: ', diff)
    return subject, 'Falta eventos: ' + str(diff) 

  print(subject.ljust(15), 'Com uma condição não prevista')
  return subject, 'Com uma condição não prevista'
  



In [15]:
subject = []
comment = []

for f in entries: 
  s , c = check(f.split('.')[0])
  subject.append(s)
  comment.append(c)

check_stim = pd.DataFrame()
check_stim['subject'] = subject
check_stim['comment'] = comment

check_stim_view = check_stim.style.set_properties(**{'text-align': 'left'})


37_run2         Tempo do Async8  2190.0 + tmax*10 deve ser menor que último sample  2190.0
15_run2         Falta eventos:  {'controle8'}
32_run2         Falta eventos:  {'controle7', 'Sync8', 'Sync7', 'controle8', 'Async7', 'Async8'}
35_run2         OK
23_run2         OK
33_run2         OK
25_run2         Tempo do Async8  2180.0 + tmax*10 deve ser menor que último sample  2190.0
2_run2          Falta eventos:  {'controle7', 'Sync8', 'controle8', 'Async7', 'Async8'}
38_run2         Falta eventos:  {'controle7', 'Sync8', 'Sync7', 'controle8', 'Async7', 'Async8'}
27_run2         OK
8_run2          OK
30_run2         OK
39_run2         Falta eventos:  {'Async7', 'Async1', 'Async4', 'Async5', 'Async6', 'Async3', 'Async2', 'Async8'}
16_run2         OK
36_run2         OK
26_run2         OK
21_run2         OK
12_run2         OK
11_run2         OK
3_run2          OK
28_run2         OK
5_run2          OK
19_run2         OK
1_run2          Falta eventos:  {'controle7', 'controle4', 'Sync3', 'Asyn

In [16]:
display(check_stim_view)

,subject,comment
0,37_run2,Tempo do Async8 + tmax*10 deve ser menor que último sample
1,15_run2,Falta eventos: {'controle8'}
2,32_run2,"Falta eventos: {'controle7', 'Sync8', 'Sync7', 'controle8', 'Async7', 'Async8'}"
3,35_run2,OK
4,23_run2,OK
5,33_run2,OK
6,25_run2,Tempo do Async8 + tmax*10 deve ser menor que último sample
7,2_run2,"Falta eventos: {'controle7', 'Sync8', 'controle8', 'Async7', 'Async8'}"
8,38_run2,"Falta eventos: {'controle7', 'Sync8', 'Sync7', 'controle8', 'Async7', 'Async8'}"
9,27_run2,OK


In [17]:
path_in

'/content/drive/MyDrive/Cond 2/snirf/'

In [18]:
check_stim.to_csv(path_in + 'check_stim_faileds.csv',index=None)